In [6]:
import re
import pandas as pd

In [16]:
df =pd.read_csv(r"C:\Users\HP\week 5\Amharic-Named-Entity-Recognition-\data\scraped_data.csv")

In [7]:
# Function to clean and tokenize Amharic text
def preprocess_text(text):
    if isinstance(text, str):  # Ensure that text is a string
        # Remove non-Amharic characters and emojis
        cleaned_text = re.sub(r'[^\u1200-\u137F ]+', '', text)  # Amharic Unicode range
        # Tokenize the text by splitting based on spaces
        tokens = cleaned_text.split()
        return tokens
    else:
        return []  # Return an empty list if the message is not a string

# Apply the preprocessing function
df['tokens'] = df['message'].apply(preprocess_text)

Labeled data saved successfully.


In [18]:
import pandas as pd
import re

# Load your dataset
df = pd.read_csv(r"C:\Users\HP\week 5\Amharic-Named-Entity-Recognition-\data\scraped_data.csv")

# Define a function to preprocess the text
def preprocess_text(text):
    if isinstance(text, str):  # Ensure the text is a string
        # Remove non-Amharic characters and emojis, keep spaces
        cleaned_text = re.sub(r'[^ሀ-ቴᎀ-ᎏ\u1200-\u137F ]+', '', text)
        # Tokenize the text by splitting based on spaces
        tokens = cleaned_text.split()
        return tokens
    return []  # Return empty if it's not a string

# Apply the preprocessing function to the 'message' column
df['tokens'] = df['message'].apply(preprocess_text)

# Preview the cleaned dataset
df[['message', 'tokens']].head()


,message,tokens
0,📣 🔠🔠🔠🔠🔠🔠🔠🔠🔠🔠\n\n📎 ይሄንን👉 t.me/MerttEka 👈 ተጭነው ...,"[ይሄንን, ተጭነው, ያድርጉ፣, ቤተሰብ]"
1,📣 MiNI HUMIDIFIER\n\n✔️ የቤት ውስጥ መዓዛ ማጨሻ\n✔️ ቤት...,"[የቤት, ውስጥ, መዓዛ, ማጨሻ, ቤትዎንቢሮዎን, በመልካም, ዘረን, ያውዱ..."
2,📣 🔠🔠🔠🔠🔠🔠🔠 Silicone Baking Mould\n\n✔️ 12 የኩኪ...,"[የኩኪስ, መስሪያ, ሞልዶች, አራት, አይነት, ሆኖ, እያንዳንዱ, ቅርፅ,..."
3,NaN,[]
4,NaN,[]


In [19]:
# Save the cleaned and tokenized dataset to a CSV file
output_path = r"C:\Users\HP\week 5\Amharic-Named-Entity-Recognition-\data\cleaned_tokenized_data.csv"
df[['message', 'tokens']].to_csv(output_path, index=False)

print(f"Cleaned and tokenized data saved to {output_path}")


Cleaned and tokenized data saved to C:\Users\HP\week 5\Amharic-Named-Entity-Recognition-\data\cleaned_tokenized_data.csv


In [4]:
import pandas as pd
import re

# Load the dataset
df = pd.read_csv(r"C:\Users\HP\week 5\Amharic-Named-Entity-Recognition-\data\scraped_data.csv")

# Function to label entities
def label_data(text):
    tokens = str(text).split()  # Split the text into tokens
    labeled_tokens = []

    for token in tokens:
        if 'ብር'in token:  # Currency
            label = 'B-PRICE'
        elif token in ['ቦሌ', 'መገናኛ','ዘፍመሽ','ግራንድ','ሰላም', 'አዲስ አበባ']:  # Locations
            label = 'B-LOC'
        elif token in ['እቃ', 'ጫማ','አበባ','ልብስ']:  # Products
            label = 'B-Product'
        else:
            label = 'O'  # Other tokens

        labeled_tokens.append((token, label))

    return labeled_tokens

# List to store the labeled data
labeled_data = []

# Process each message in the dataframe
for message in df['message']:  # Assuming 'message' is the column containing text
    if pd.notna(message):  # Check if the message is not NaN
        labeled_sentence = label_data(message)
        labeled_data.extend(labeled_sentence)
        labeled_data.append((',', 'O'))  # Add a sentence separator

# Convert the labeled data into a DataFrame
labeled_df = pd.DataFrame(labeled_data, columns=['Token', 'Label'])

# Save the labeled data to CSV
labeled_df.to_csv(r"C:\Users\HP\week 5\Amharic-Named-Entity-Recognition-\data\labeled_amharic_data.csv", index=False)

print("Labeled data saved successfully.")


Labeled data saved successfully.
